论文地址：https://arxiv.org/abs/2010.11929

MLPMixer 的原理：

![MLPMixer](./MLPMixer.png)

Flax 中 `einops.rearrange(x, 'n h w c -> n (h w) c')` 的作用和 PyTorch 中 `x.view(n, h * w, c)` 的作用相同，即将输入的张量 x 重塑为 (n, h * w, c) 形状。
如果要改成 PyTorch 中模块的形式，可以写为 `nn.Flatten(start_dim=1, end_dim=2)(x)`。

In [ ]:
import torch
import torch.nn as nn

import einops

In [1]:
x = torch.randn(2, 3, 224, 224)
x = nn.Flatten(start_dim=1, end_dim=2)(x)
print(x.shape)

torch.Size([2, 672, 224])


In [2]:
x = torch.randn(2, 3, 224, 224)
x = einops.rearrange(x, 'n h w c -> n (h w) c')
print(x.shape)

torch.Size([2, 672, 224])


In [4]:
x = torch.randn(2, 3, 224, 224)
x = x.view(2, -1, 224)
print(x.shape)

torch.Size([2, 672, 224])


MLP-Mixer即可以靠channel-mixing MLPs层结合不同channels的信息，也可以靠token-mixing MLPs层结合不同空间位置的信息。

CNN的特点是inductive bias，ViT靠大量数据(JFT-300数据集)使性能战胜了CNN，说明大量的数据是可以战胜inductive bias的，这个MLP-Mixer也是一样。卷积相当于是一种认为设计的学习模式：即局部假设。能够以天然具备学习相邻信息的优势，但长远看来，在数据和算力提升的前提下，相比于attention甚至MLP，可能成为了限制。因为不用滑窗，也不用attention的方法其实是CNN的母集。

早起人们放弃MLP而使用CNN的原因是算力不足，CNN更节省算力，训练好模型更容易。现在算力资源提高了，就有了重新回到MLP的可能。MLP-Mixer说明在分类这种简单的任务上是可以通过算力的堆砌来训练出比CNN更广义的MLP模型 (CNN可以看做是狭义的MLP)。

最后，channel-mixing MLPs层相当于1×1 convolution，而token-mixing MLPs层相当于广义的depth-wise convolution，只是MLP-Mixer让这两种类型的层交替执行了。